Please use the same environment installed 

In [1]:
from pathlib import Path
from rio_tools import get_geopandas_features_from_array
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr

/Users/cmarshak/miniforge3/envs/topsapp_env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
INDEX_TO_CHECK = 0

In [13]:
nc_paths_new = sorted(list(Path('new_products/').glob('*.nc')))
nc_paths_new

[PosixPath('new_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc'),
 PosixPath('new_products/S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1.nc')]

In [14]:
nc_paths_old = sorted(list(Path('existing_products/').glob('*.nc')))
nc_paths_old

[PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240208_20240103-015044-00119W_00033N-PP-629e-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240208_20240115-015044-00119W_00033N-PP-8b54-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240115-015044-00119W_00033N-PP-60cc-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1.nc')]

In [15]:
new_prod = nc_paths_new[INDEX_TO_CHECK]
new_prod

PosixPath('new_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc')

In [16]:
len('S1-GUNW-A-R-064-tops-20240220_20240127')

38

In [17]:
old_prods = sorted(list(Path('existing_products/').glob(f'{new_prod.name[:38]}*.nc')))
assert len(old_prods) == 1
old_prod = old_prods[0]
old_prod

PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc')

In [18]:
with rasterio.open(f'{str(new_prod)}') as ds:
    subdatasets = ds.subdatasets
    print(ds.transform)
subdatasets = sorted([sd for sd in subdatasets if 'productBoundingBox' not in sd])
subdatasets[:2]

| 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|


['netcdf:new_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc:/science/grids/corrections/derived/ionosphere/ionosphere',
 'netcdf:new_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc:/science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps']

In [19]:
local_vars = [sd.split(':')[-1] for sd in subdatasets]
local_vars[:2]

['/science/grids/corrections/derived/ionosphere/ionosphere',
 '/science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps']

In [21]:
for golden_path, local_var in zip(subdatasets, local_vars):
    print('#'*10)
    print(local_var)
    print('#'*10)
    with rasterio.open(f'netcdf:{str(old_prod)}:{local_var[1:]}') as ds:
        X_golden = ds.read()
        trans_golden = ds.transform
    with rasterio.open(f'netcdf:{str(new_prod)}:{local_var[1:]}') as ds:
        X_new = ds.read()
        trans_new = ds.transform

    if local_var == '/science/grids/corrections/derived/ionosphere/ionosphere':
        iono_new = X_new
        iono_old = X_golden
        

    X_new_data = X_new.ravel()
    X_golden_data = X_golden.ravel()
    if X_new.dtype in [np.float32, np.float64]:
        X_new_data = X_new[~np.isnan(X_new)]
        X_golden_data = X_golden[~np.isnan(X_golden)]

    print(f'The variable {local_var} has RMS error: ', np.linalg.norm(X_new_data - X_golden_data))
    print(f'The origin of the new dataset is: ', trans_new.c, trans_new.f)
    print(f'The origin of the g(old)en dataset is: ', trans_golden.c, trans_golden.f)
    print('\n')

##########
/science/grids/corrections/derived/ionosphere/ionosphere
##########
The variable /science/grids/corrections/derived/ionosphere/ionosphere has RMS error:  3.6565418
The origin of the new dataset is:  -119.024167057 34.889267111
The origin of the g(old)en dataset is:  -119.024167057 34.889267111


##########
/science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps
##########
The variable /science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps has RMS error:  2.2352712
The origin of the new dataset is:  -119.024167057 34.889267111
The origin of the g(old)en dataset is:  -119.024167057 34.889267111


##########
/science/grids/corrections/external/tides/solidEarth/reference/solidEarthTide
##########
The variable /science/grids/corrections/external/tides/solidEarth/reference/solidEarthTide has RMS error:  0.0
The origin of the new dataset is:  -119.25 35.35
The origin of the g(old)en dataset is:  -119.25 35.35


##########
/science/grids/co